In [1]:
import pandas as pd
import numpy as np
from trading_rule import *
from trading_model import *
import matplotlib.pyplot as plt
from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database
import psycopg2 as pg

# Data

In [2]:
from mt5_interface import User
from postgres_interface import DataBase
import json
from data_collection import PretradeDataUpdater

In [3]:
def read_json(filename):
  with open(filename) as f:
    j = json.load(f)
  return j

ac_info = read_json('config/mt5_account.json')['mt5-demo']
db_info = read_json('config/postgres_info.json')['pre-trade']

In [4]:
update = PretradeDataUpdater(ac_info,db_info)

Logged in as 80273160
Connected to pre-trade


In [5]:
update.set_universe()

In [6]:
df = pd.DataFrame()
df["symbol"] = ["AUDUSD", "EURUSD", "GBPUSD", "NZDUSD", "USDCAD", "USDJPY"]
df["description"] = [
    "Australian Dollar (in USD)",
    "Euro (in USD)",
    "UK Pound (in USD)",
    "New Zealand Dollar (in USD)",
    "US dollar (in CAD)",
    "US dollar (in JPY)",
]
df["asset_class"] = "currency"
df["nominal_currency"] = ["USD", "USD", "USD", "USD", "CAD", "JPY"]
df["id"] = range(len(df))
df = df[["id", "symbol", "asset_class", "nominal_currency", "description"]]
df

,id,symbol,asset_class,nominal_currency,description
0,0,AUDUSD,currency,USD,Australian Dollar (in USD)
1,1,EURUSD,currency,USD,Euro (in USD)
2,2,GBPUSD,currency,USD,UK Pound (in USD)
3,3,NZDUSD,currency,USD,New Zealand Dollar (in USD)
4,4,USDCAD,currency,CAD,US dollar (in CAD)
5,5,USDJPY,currency,JPY,US dollar (in JPY)


In [7]:
update.db.drop('bars')

In [8]:
update.init_bars()
update.db.read_all('bars')

Getting 1h bars of AUDUSD from 2013-04-12 10:54:35.637365+00:00 to 2024-03-25 10:54:35.637365+00:00
Getting 1h bars of EURUSD from 2013-04-12 10:54:45.738806+00:00 to 2024-03-25 10:54:45.738806+00:00
Getting 1h bars of GBPUSD from 2013-04-12 10:54:55.319818+00:00 to 2024-03-25 10:54:55.319818+00:00
Getting 1h bars of NZDUSD from 2013-04-12 10:55:05.016818+00:00 to 2024-03-25 10:55:05.016818+00:00
Getting 1h bars of USDCAD from 2013-04-12 10:55:14.671819+00:00 to 2024-03-25 10:55:14.671819+00:00
Getting 1h bars of USDJPY from 2013-04-12 10:55:24.287806+00:00 to 2024-03-25 10:55:24.287806+00:00


,real_volume,spread,tick_volume,time,open,high,low,close,symbol,id
0,2001-20130412110000,AUDUSD,2013-04-12 11:00:00,1.05397,1.05402,1.05148,1.05219,1,2146,0
1,2001-20130412120000,AUDUSD,2013-04-12 12:00:00,1.05217,1.05315,1.05205,1.05286,6,1202,0
2,2001-20130412130000,AUDUSD,2013-04-12 13:00:00,1.05287,1.05311,1.05224,1.05284,7,1158,0
3,2001-20130412140000,AUDUSD,2013-04-12 14:00:00,1.05284,1.05338,1.05111,1.05301,5,2561,0
4,2001-20130412150000,AUDUSD,2013-04-12 15:00:00,1.05307,1.05372,1.05184,1.05296,1,1876,0
...,...,...,...,...,...,...,...,...,...,...
407599,2006-20240325060000,USDJPY,2024-03-25 06:00:00,151.23300,151.37500,151.23300,151.36600,1,1785,0
407600,2006-20240325070000,USDJPY,2024-03-25 07:00:00,151.35900,151.37200,151.19200,151.21500,0,1357,0
407601,2006-20240325080000,USDJPY,2024-03-25 08:00:00,151.21500,151.32100,151.17500,151.32000,0,1741,0
407602,2006-20240325090000,USDJPY,2024-03-25 09:00:00,151.31500,151.38500,151.16800,151.32100,0,2381,0


In [7]:
update.update_bars()

real_volume    2001-20240325100000
spread                      AUDUSD
tick_volume    2024-03-25 10:00:00
time                       0.65216
open                       0.65302
high                       0.65205
low                         0.6528
close                            1
symbol                        1368
id                               0
dtype: object
<class 'float'>
real_volume    2002-20240325100000
spread                      EURUSD
tick_volume    2024-03-25 10:00:00
time                       1.08139
open                       1.08237
high                       1.08071
low                        1.08137
close                            0
symbol                        1832
id                               0
dtype: object
<class 'float'>
real_volume    2003-20240325100000
spread                      GBPUSD
tick_volume    2024-03-25 10:00:00
time                       1.26023
open                       1.26166
high                       1.25951
low                        1.2

In [10]:
update.close()

In [12]:
update.user.get_bars('AUDUSD','1h',period_days=100)

Getting 1h bars of AUDUSD from 2023-12-16 09:57:54.734045+00:00 to 2024-03-25 09:57:54.734045+00:00


,time,open,high,low,close,tick_volume,spread,real_volume,time_value
0,2023-12-18 00:00:00,0.66970,0.67030,0.66947,0.66981,1578,14,0,1702857600
1,2023-12-18 01:00:00,0.66985,0.67031,0.66929,0.66983,1599,4,0,1702861200
2,2023-12-18 02:00:00,0.66982,0.67090,0.66978,0.67079,2536,4,0,1702864800
3,2023-12-18 03:00:00,0.67079,0.67109,0.66928,0.66994,3190,4,0,1702868400
4,2023-12-18 04:00:00,0.66994,0.67094,0.66994,0.67061,2271,0,0,1702872000
...,...,...,...,...,...,...,...,...,...
1638,2024-03-25 05:00:00,0.65408,0.65409,0.65252,0.65270,1167,2,0,1711342800
1639,2024-03-25 06:00:00,0.65270,0.65298,0.65250,0.65265,753,3,0,1711346400
1640,2024-03-25 07:00:00,0.65265,0.65276,0.65231,0.65254,1040,1,0,1711350000
1641,2024-03-25 08:00:00,0.65255,0.65257,0.65202,0.65217,1328,1,0,1711353600


In [ ]:
symbols = ['AUDUSD','EURUSD','GBPUSD','NZDUSD','USDCAD','USDJPY']
for symbol in symbols:
  df = ac.get_bars(symbol,'1h',period_days=4000)
  db.create_table_from_df(table_name=symbol,df=df)
db.commit()
db.close()

In [4]:
ac = User().login_by_dict(ac_info)

Logged in as 80273160


In [5]:
ac.get_bars('XAUUSD','1h',period_days=4000)

Getting 1h bars of XAUUSD from 2013-04-12 07:30:46.262861+00:00 to 2024-03-25 07:30:46.262861+00:00


,time,open,high,low,close,tick_volume,spread,real_volume
0,2013-04-12 08:00:00,1560.99,1564.71,1560.80,1563.53,3806,23,946000
1,2013-04-12 09:00:00,1563.53,1563.63,1560.80,1561.55,4039,20,1096700
2,2013-04-12 10:00:00,1561.59,1561.63,1556.94,1557.04,6341,21,1685990
3,2013-04-12 11:00:00,1557.04,1558.05,1556.88,1557.20,4950,18,1307760
4,2013-04-12 12:00:00,1557.19,1557.19,1545.10,1545.63,13158,17,3626680
...,...,...,...,...,...,...,...,...
64408,2024-03-25 03:00:00,2165.49,2176.90,2164.03,2175.14,5505,5,0
64409,2024-03-25 04:00:00,2175.16,2178.29,2173.77,2175.05,3268,5,0
64410,2024-03-25 05:00:00,2175.05,2175.09,2170.03,2170.60,3001,5,0
64411,2024-03-25 06:00:00,2170.60,2171.56,2169.13,2169.41,1832,7,0


In [12]:
db = DataBase()
db.connect_by_dict(db_info)

Connected to bars


In [13]:
symbols = ['AUDUSD','EURUSD','GBPUSD','NZDUSD','USDCAD','USDJPY']
for symbol in symbols:
  df = ac.get_bars(symbol,'1h',period_days=4000)
  db.create_table_from_df(table_name=symbol,df=df)
db.commit()
db.close()

Getting 1h bars of AUDUSD from 2013-04-09 14:29:25.679662+00:00 to 2024-03-22 14:29:25.679662+00:00
Getting 1h bars of EURUSD from 2013-04-09 14:29:36.198662+00:00 to 2024-03-22 14:29:36.198662+00:00
Getting 1h bars of GBPUSD from 2013-04-09 14:29:46.394661+00:00 to 2024-03-22 14:29:46.394661+00:00
Getting 1h bars of NZDUSD from 2013-04-09 14:29:56.607660+00:00 to 2024-03-22 14:29:56.607660+00:00
Getting 1h bars of USDCAD from 2013-04-09 14:30:06.861662+00:00 to 2024-03-22 14:30:06.861662+00:00
Getting 1h bars of USDJPY from 2013-04-09 14:30:17.188661+00:00 to 2024-03-22 14:30:17.188661+00:00


In [11]:
db.close()

[('time',),
 ('open',),
 ('high',),
 ('low',),
 ('close',),
 ('tick_volume',),
 ('spread',),
 ('real_volume',),
 ('id',)]

In [10]:
db.commit()
db.close()

In [7]:
df.dtypes

time           datetime64[ns]
open                  float64
high                  float64
low                   float64
close                 float64
tick_volume            uint64
spread                  int32
real_volume            uint64
id                     object
dtype: object

In [ ]:
symbols = ['AUDUSD','EURUSD','GBPUSD','NZDUSD','USDCAD','USDJPY']
for symbol in symbols:
  df = ac.get_bars(symbol,'1h',period_days=4000)
  

In [6]:
db.read_tail('audusd',10)

UndefinedColumn: column "createddatetime" does not exist
LINE 2: SELECT * FROM audusd ORDER BY CreatedDateTime DESC LIMIT 10;
                                      ^


In [7]:
db.close()

In [ ]:
data_path = f'Data/Forex/GBPUSD.csv'
data = pd.read_csv(data_path)

In [ ]:
data.set_index('Datetime',inplace=True)
data.sort_index(ascending=True,inplace=True)

# Models

In [ ]:
signal = []
train_size = 12000
start = 1000
for i in range(0+start,24+start):
  train_set = data.iloc[i:train_size+i]
  
  model = ShortTermBollingerBand(train_set.Open,
                         train_set.High,
                         train_set.Low,
                         train_set.Close,
                         train_set.Volume)
  signal.append(model.signal)

In [ ]:
pos = pd.Series(signal,index=data.index[train_size+start:train_size+start+24])
logret = data.Close.diff().shift(-1)
pnl = pos*logret
pnl.dropna().cumsum().plot()


# Rules

In [ ]:
def store_rule(rule_obj:TradingRule,signals,bisignals,pnls):
  signals.append(rule_obj.signal)
  bisignals.append(rule_obj.bisignal)
  pnls.append(rule_obj.pnl)

In [ ]:
from itertools import product
signals = []
bisignals = []
pnls = []

lookbacks = [12*2**i for i in range(6)]

for lookback in lookbacks:
  for thr in [0.85,0.95,0.99]:
    store_rule(ChannelBreakout(lookback,thr,
                    data.Open,data.High,data.Low,data.Close,data.Volume),
              signals,bisignals,pnls)
    store_rule(SkewPremium(lookback,thr,
                    data.Open,data.High,data.Low,data.Close,data.Volume),
              signals,bisignals,pnls)
    store_rule(KurtReversal(lookback,thr,
                    data.Open,data.High,data.Low,data.Close,data.Volume),
              signals,bisignals,pnls)
  for thr in [1,2,3]:
    store_rule(EWMAC(lookback,thr,
                    data.Open,data.High,data.Low,data.Close,data.Volume),
              signals,bisignals,pnls)
    store_rule(BollingerBand(lookback,thr,
                    data.Open,data.High,data.Low,data.Close,data.Volume),
              signals,bisignals,pnls)
    store_rule(NDayMomentum(lookback,thr,
                    data.Open,data.High,data.Low,data.Close,data.Volume),
              signals,bisignals,pnls)
  

In [ ]:
signals_df = pd.concat(signals,axis=1).dropna()
bisignals_df = pd.concat(bisignals,axis=1).dropna()
pnls_df = pd.concat(pnls,axis=1).dropna()

In [ ]:
def sr(pnl):
  pnl = pnl.fillna(0)
  mu = pnl.mean()
  sig = pnl.std()
  n_per_year = 24*252
  sr = mu/sig*n_per_year**0.5
  return sr

In [ ]:
strats = ['EWMAC','ChannelBreakout','NDayMomentum','BollingerBand','SkewPremium','KurtReversal']

fig,axes = plt.subplots(2,3,figsize=(30,16))
for i,strat in enumerate(strats):
  cols = pnls_df.columns[pnls_df.columns.str.match(f'{strat}_*')]
  ax = axes[i//3,i%3]
  pnls_df.loc[:,cols].cumsum().plot(ax=ax)

# KMean

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
n_clusters = 12
kmeans = KMeans(n_clusters=n_clusters, random_state=0, n_init="auto")
kmeans.fit(pnls_df.corr())
groups = kmeans.labels_

port = {i: list() for i in range(n_clusters)}
for i,group in enumerate(groups):
  port[group].append(pnls_df.columns[i])

In [ ]:
fig,axes = plt.subplots(4,3,figsize=(30,32))
for i,s_list in (port.items()):
  ax = axes[i//3,i%3]
  pnls_df.loc[:,s_list].cumsum().plot(ax=ax)

In [ ]:
import pickle


def save_pickle(obj, filename):
    with open(f"{filename}.pickle", "wb") as handle:
        pickle.dump(obj, handle, protocol=pickle.HIGHEST_PROTOCOL)
def load_pickle(filename):
    with open(f"{filename}.pickle", "rb") as handle:
        obj = pickle.load(handle)
        return obj

In [ ]:
save_pickle(port,'strat_cluster')

In [ ]:
port = load_pickle('strat_cluster')

# Labelling

In [ ]:
def get_label(prc, tp, sl, hd):
    log_prc = np.log(prc)
    log_ret = log_prc.diff().shift(-1)
    ew_vol = np.sqrt((log_ret**2).ewm(40, min_periods=40).mean())

    # tp = window**0.5
    # sl = window**0.5
    # hd = window

    # print(tp , sl , hd)
    label = []

    for i, ret in log_ret.items():
        up_bar = ew_vol.loc[i] * tp
        low_bar = ew_vol.loc[i] * -sl
        time_bar = hd

        future_ret = log_ret.loc[i:]
        cum_ret = ret
        cum_day = 1
        while True:
            if cum_ret >= up_bar:
                label.append(+2)
                break
            elif cum_ret <= low_bar:
                label.append(np.sign(cum_ret))
                break
            elif cum_day >= time_bar or cum_day >= future_ret.shape[0]:
                label.append(-2)
                break

            ret = future_ret.iloc[cum_day]
            cum_day += 1
            cum_ret += ret

    label = pd.Series(label, index=log_ret.index, name="LABEL")
    return label

In [ ]:
tp,sl,hd = 5,5,24
label = get_label(data.Close,tp,sl,hd)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from statsmodels.api import OLS

from numpy.lib.stride_tricks import sliding_window_view

In [ ]:
def get_pred(dataset):
    #dataset = dataset[0,0]
    #print(dataset)
    train_set = dataset[:, :-1]
    test_set = dataset[:, -1]
    train_y, train_x = train_set[0], train_set[1:]
    test_y, test_x = test_set[0], test_set[1:]
    #model = RandomForestClassifier(n_estimators=500, max_depth=3, random_state=300300)
    model = GaussianNB()
    #model = LogisticRegression(penalty='l2',random_state=300300)
    model.fit(y=train_y, X=train_x.T)
    pred = model.predict(test_x.reshape(1, -1))[0]
    return pred

In [ ]:
p = pd.concat([label,bisignals_df.loc[:,port[3]]],axis=1).dropna()
lookback = 9600

# create rolling datasets for each day
rolling = sliding_window_view(p,axis=0, window_shape=lookback+1)
# random forest prediction
#pred = np.apply_over_axis(get_pred, axis=1, arr=rolling)
size = 70000
pred = [get_pred(dataset) for dataset in rolling[:size]]
pred_d = pd.Series(pred,p.index[lookback+1:lookback+1+size])

In [ ]:
pred_d = pd.Series(pred,p.index[lookback:lookback+size])

In [ ]:
pred_d

In [ ]:
sr(pred_d * data.Close.diff().shift(-1))

In [ ]:
sr(pnls_df.loc[:,port[3]].mean(axis=1))

In [ ]:
(pred_d/2 * data.Close.diff().shift(-1)).dropna().cumsum().plot()
(pnls_df.loc[:,port[9]].mean(axis=1)).cumsum().plot()
data.Close.diff().shift(-1).cumsum().plot()

In [ ]:
pred_d = pd.Series(pred,p.index[2401:2401+5000])
pred_d.tail()

In [ ]:
vf = np.vectorize(lambda x: print(x))
vf(rolling[:1])

In [ ]:
vget_pred = np.vectorize(get_pred)
vget_pred(rolling[:2])

In [ ]:
r1[:,-10:]

In [ ]:
p.iloc[2391:2401]

# AggClust

In [ ]:
from scipy.cluster.hierarchy import dendrogram
from sklearn.cluster import AgglomerativeClustering

In [ ]:
def plot_dendrogram(model,ax):
    # Create linkage matrix and then plot the dendrogram

    # create the counts of samples under each node
    counts = np.zeros(model.children_.shape[0])
    n_samples = len(model.labels_)
    for i, merge in enumerate(model.children_):
        current_count = 0
        for child_idx in merge:
            if child_idx < n_samples:
                current_count += 1  # leaf node
            else:
                current_count += counts[child_idx - n_samples]
        counts[i] = current_count

    linkage_matrix = np.column_stack(
        [model.children_, model.distances_, counts]
    ).astype(float)

    # Plot the corresponding dendrogram
    dendrogram(linkage_matrix,ax=ax)

In [ ]:
model = AgglomerativeClustering(distance_threshold=0, n_clusters=None)
model.fit(pnls_df.corr())

In [ ]:
model.n_clusters_

In [ ]:
fig,ax = plt.subplots(figsize=(40,16))
plot_dendrogram(model,ax=ax)

# HRP

In [ ]:
from pypfopt.hierarchical_portfolio import HRPOpt

In [ ]:
hrp = HRPOpt(pnls_df)

In [ ]:
hrp.optimize()

In [ ]:
hrp.portfolio_performance(risk_free_rate=0,frequency=252*24)